In [39]:
import pandas as pd
import numpy as np
from tqdm import tqdm
df = pd.read_csv("/datasets/quora/train.csv")
df = df.head(10000)

In [2]:
df[df.isnull().any(axis=1)]

,id,qid1,qid2,question1,question2,is_duplicate
105780,105780,174363,174364,How can I develop android app?,NaN,0
201841,201841,303951,174364,How can I create an Android app?,NaN,0
363362,363362,493340,493341,NaN,My Chinese name is Haichao Yu. What English na...,0


In [3]:
df.drop(df[df.isnull().any(axis=1)].index,inplace=True)

In [4]:
import re
def text_to_word_list(text , split):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    #split to words for word2vec
    if split :
        text = text.split()

    return text

In [40]:
question1 = []
question2 = []
question1TF = []
question2TF = []
for x in df["question1"]:
    question1.append(text_to_word_list(x, True))
    question1TF.append(text_to_word_list(x, False))
for x in df["question2"]:
    question2.append(text_to_word_list(x, True))
    question2TF.append(text_to_word_list(x, False))

In [17]:
from gensim.models import word2vec

questions = question1 + question2

# train model
model = word2vec.Word2Vec(questions, size=10, min_count=1)

In [18]:
model.save('modelw2v.bin')

In [19]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('modelw2v.bin')

In [20]:
# exctract word2vec vectors
vecs1 = []
for qu in question1:
    word_vec = []
    for word in qu:
        # word2vec
        word_vec.append(model.wv[word])
    vecs1.append(word_vec)
df['q1_feats'] = list(vecs1)

In [21]:
vecs2 = []
for qu2 in question2:
    word_vec2 = []
    for word in qu2:
        # word2vec
        word_vec2.append(model.wv[word])
    vecs2.append(word_vec2)
df['q2_feats'] = list(vecs2)

In [22]:
df

,id,qid1,qid2,question1,question2,is_duplicate,q1_feats,q2_feats
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[[1.7473717, -2.6262953, -0.10161054, -0.87554...","[[1.7473717, -2.6262953, -0.10161054, -0.87554..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[[1.7473717, -2.6262953, -0.10161054, -0.87554...","[[1.7473717, -2.6262953, -0.10161054, -0.87554..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[[-2.3355067, 2.0692139, -0.8257057, 0.3185072...","[[-2.3355067, 2.0692139, -0.8257057, 0.3185072..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[[-0.16869572, 3.5913603, 0.35479644, 0.276689...","[[6.995065, 3.4047115, -2.1689236, 2.414899, -..."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[[2.1625907, -2.6822379, 2.4315636, -0.7801787...","[[2.1625907, -2.6822379, 2.4315636, -0.7801787..."
...,...,...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0,"[[-2.3355067, 2.0692139, -0.8257057, 0.3185072...","[[-2.3355067, 2.0692139, -0.8257057, 0.3185072..."
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1,"[[-2.1759653, 1.8239492, 0.89487946, 1.6334667...","[[-0.86929, -1.8171616, 0.43591565, -0.6005701..."
404287,404287,537928,537929,What is one coin?,What's this coin?,0,"[[1.7473717, -2.6262953, -0.10161054, -0.87554...","[[1.7473717, -2.6262953, -0.10161054, -0.87554..."
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0,"[[1.7473717, -2.6262953, -0.10161054, -0.87554...","[[-3.9164126, 4.0500646, 0.73007447, 1.7626686..."


In [23]:
print(df['q2_feats'][0])

[array([ 1.7473717 , -2.6262953 , -0.10161054, -0.8755403 , -1.1663448 ,
        0.8059146 ,  2.0686438 ,  1.2114148 ,  0.88742393,  0.45057452],
      dtype=float32), array([-0.86929   , -1.8171616 ,  0.43591565, -0.6005701 , -0.38657954,
       -0.67453945,  0.24090974, -0.32868263,  1.8809551 ,  0.4018846 ],
      dtype=float32), array([-0.53186184, -1.4571234 ,  1.7137733 , -1.5512898 ,  0.05082732,
       -0.17809573,  0.3911812 ,  2.0384965 ,  0.7645675 ,  0.3621754 ],
      dtype=float32), array([-0.48199895,  3.2713783 ,  2.0729063 , -1.4789608 , -0.03389227,
        0.34500214,  1.5566995 ,  2.2249734 ,  0.25469473, -4.3693867 ],
      dtype=float32), array([-2.2341025 ,  2.940374  ,  4.9096627 , -0.02052404,  0.8326176 ,
       -3.298926  , -1.1585256 ,  0.69731516,  1.6589822 ,  3.3604858 ],
      dtype=float32), array([-0.48199895,  3.2713783 ,  2.0729063 , -1.4789608 , -0.03389227,
        0.34500214,  1.5566995 ,  2.2249734 ,  0.25469473, -4.3693867 ],
      dtype=float32

In [41]:
questionsTF = question1TF + question2TF
question1TF

['what is the step by step guide to invest in share market in india ',
 'what is the story of kohinoor koh - i - noor diamond ',
 'how can i increase the speed of my internet connection while using a vpn ',
 'why am i mentally very lonely how can i solve it ',
 'which one dissolve in water quikly sugar salt methane and carbon di oxide ',
 'astrology : i am a capricorn sun cap moon and cap rising what does that say about me ',
 'should i buy tiago ',
 'how can i be a good geologist ',
 'when do you use instead of ',
 'motorola company : can i hack my charter motorolla dcx3400 ',
 'method to find separation of slits using fresnel biprism ',
 'how do i read and find my youtube comments ',
 'what can make physics easy to learn ',
 'what was your first sexual experience like ',
 'what are the laws to change your status from a student visa to a green card in the us how do they compare to the immigration laws in canada ',
 'what would a trump presidency mean for current international master s

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit_transform(question1TF)

<10000x11714 sparse matrix of type '<class 'numpy.float64'>'
	with 99975 stored elements in Compressed Sparse Row format>

In [44]:
print(tfidf.vocabulary_)
print(tfidf.idf_)

{'what': 11450, 'is': 5499, 'the': 10493, 'step': 9978, 'by': 1706, 'guide': 4626, 'to': 10627, 'invest': 5445, 'in': 5187, 'share': 9421, 'market': 6374, 'india': 5227, 'story': 10025, 'of': 7261, 'kohinoor': 5816, 'koh': 5815, 'noor': 7140, 'diamond': 3068, 'how': 4980, 'can': 1762, 'increase': 5215, 'speed': 9829, 'my': 6914, 'internet': 5396, 'connection': 2410, 'while': 11469, 'using': 11083, 'vpn': 11310, 'why': 11484, 'am': 695, 'mentally': 6546, 'very': 11195, 'lonely': 6158, 'solve': 9742, 'it': 5528, 'which': 11468, 'one': 7310, 'dissolve': 3196, 'water': 11376, 'quikly': 8402, 'sugar': 10147, 'salt': 9125, 'methane': 6589, 'and': 755, 'carbon': 1809, 'di': 3056, 'oxide': 7483, 'astrology': 1003, 'capricorn': 1799, 'sun': 10169, 'cap': 1789, 'moon': 6789, 'rising': 8937, 'does': 3257, 'that': 10491, 'say': 9178, 'about': 346, 'me': 6474, 'should': 9499, 'buy': 1701, 'tiago': 10572, 'be': 1237, 'good': 4510, 'geologist': 4420, 'when': 11461, 'do': 3247, 'you': 11673, 'use': 11